In [4]:
#!sudo apt-get install libpq-dev -y
!pip3 install psycopg2

# General psql queries

In [5]:
import psycopg2

In [9]:
#открываем соединение
conn_string="dbname='{}' user='{}' host='{}' password='{}'".format("titanik_db8", "student8",
                                                                  "ec2-18-202-17-151.eu-west-1.compute.amazonaws.com",
                                                                  "student8")
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Создаем таблицы в базе

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("../lecture_05_ml/notebooks/titanik.tsv", sep='|', index_col=0)
data['prename'] = data['name'].apply(lambda x: x.split()[1].strip("."))
data['pclass'] = data['pclass'].apply(str)
data['age'].fillna(0, inplace=True)
data['age'] = data['age'].apply(float)
data['fare'].fillna(0, inplace=True)
data['fare'] = data['fare'].apply(float)
data['embarked'].fillna("None", inplace=True)

lbl_embarked = LabelEncoder().fit(data['embarked'])
lbl_pclass = LabelEncoder().fit(data['pclass'])
lbl_prename = LabelEncoder().fit(data['prename'])
lbl_sex = LabelEncoder().fit(data['sex'])

data['embarked'] = lbl_embarked.transform(data['embarked'])
data['pclass'] = lbl_pclass.transform(data['pclass'])
data['prename'] = lbl_prename.transform(data['prename'])
data['sex'] = lbl_sex.transform(data['sex'])

data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,prename
0,0,1,"Allen, Miss. Elisabeth Walton",0,29.0000,0,0,24160,211.3375,B5,3,2,NaN,"St Louis, MO",16
1,0,1,"Allison, Master. Hudson Trevor",1,0.9167,1,2,113781,151.5500,C22 C26,3,11,NaN,"Montreal, PQ / Chesterville, ON",13
2,0,0,"Allison, Miss. Helen Loraine",0,2.0000,1,2,113781,151.5500,C22 C26,3,NaN,NaN,"Montreal, PQ / Chesterville, ON",16
3,0,0,"Allison, Mr. Hudson Joshua Creighton",1,30.0000,1,2,113781,151.5500,C22 C26,3,NaN,135.0,"Montreal, PQ / Chesterville, ON",19
4,0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",0,25.0000,1,2,113781,151.5500,C22 C26,3,NaN,NaN,"Montreal, PQ / Chesterville, ON",20


In [19]:
# Создадим впомогательные таблицы и заполним их
create_additional = '''
        CREATE TABLE embarked (
        embarked_id serial PRIMARY KEY,
        embarked_type VARCHAR(10) NOT NULL UNIQUE
        );
        CREATE TABLE pclass (
        pclass_id serial PRIMARY KEY,
        pclass_type VARCHAR(10) NOT NULL UNIQUE
        );
        '''
cursor.execute(create_additional)
conn.commit()

In [63]:
conn.commit()

In [21]:
insert_query = '''INSERT INTO embarked (embarked_id, embarked_type) VALUES (%s, %s)'''
cursor.executemany(insert_query, list(zip(range(len(lbl_embarked.classes_)), lbl_embarked.classes_)))
conn.commit()
insert_query = '''INSERT INTO pclass (pclass_id, pclass_type) VALUES (%s, %s)'''
cursor.executemany(insert_query, list(zip(range(len(lbl_pclass.classes_)), lbl_pclass.classes_)))
conn.commit()

In [35]:
# Создайте таблицу для prename (и заполните её)
create_additional = '''
        CREATE TABLE prename (
        prename_id serial PRIMARY KEY,
        prename_type VARCHAR(30) NOT NULL
        )
        '''
cursor.execute(create_additional)
conn.commit()

DuplicateTable: relation "prename" already exists


In [38]:
change = """
    ALTER TABLE prename ALTER COLUMN prename_type TYPE VARCHAR(30)
"""
cursor.execute(change)
conn.commit()

In [39]:
insert_query = '''INSERT INTO prename (prename_id, prename_type) VALUES (%s, %s)'''
cursor.executemany(insert_query, list(zip(range(len(lbl_prename.classes_)), lbl_prename.classes_)))
conn.commit()

In [43]:
# Создайте таблицу для пола
create_additional = '''
        CREATE TABLE sex (
        sex_id serial PRIMARY KEY,
        sex_type VARCHAR(10) NOT NULL
        )
        '''
cursor.execute(create_additional)
conn.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [45]:
insert_query = '''INSERT INTO sex (sex_id, sex_type) VALUES (%s, %s)'''
cursor.executemany(insert_query, list(zip(range(len(lbl_sex.classes_)), lbl_sex.classes_)))
conn.commit()

In [46]:
data.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest', 'prename'],
      dtype='object')

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 15 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null int64
age          1309 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1309 non-null float64
cabin        295 non-null object
embarked     1309 non-null int64
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
prename      1309 non-null int64
dtypes: float64(3), int64(7), object(5)
memory usage: 163.6+ KB


In [51]:
# создать основную таблицу
create_additional = '''
        CREATE TABLE titanik (
        sex INT4 REFERENCES sex(sex_id)
        )
        '''
cursor.execute(create_additional)
conn.commit()

In [60]:
from random import randint
insert_query = '''INSERT INTO titanik (sex) VALUES (%s)'''
cursor.executemany(insert_query, [(randint(0,1),) for _ in range(19)])
conn.commit()

In [64]:
create_main = '''
    CREATE TABLE titanik1 (
        record_id serial PRIMARY KEY,
        pclass_id INTEGER REFERENCES pclass(pclass_id),
        survived INT4 NOT NULL,
        name varchar(200),
        sex INTEGER REFERENCES sex(sex_id),
        age FLOAT,
        sibsp INTEGER,
        parch INTEGER,
        ticket VARCHAR(100),
        fare FLOAT,
        cabin VARCHAR(100),
        embarked INTEGER REFERENCES embarked(embarked_id),
        boat VARCHAR(100),
        body VARCHAR(100),
        dest VARCHAR(100),
        prename_id INTEGER REFERENCES prename(prename_id)
    );
    '''
cursor.execute(create_main)
conn.commit()

In [68]:
insert_query = '''
INSERT INTO titanik1 (pclass_id, survived, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked, boat, body, dest, prename_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''
cursor.executemany(insert_query, data.values[:10])
conn.commit()

In [67]:
conn.commit()

In [53]:
[randint(0,1) for _ in range(20)]

[0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0]

In [73]:
#TODO Вопрос Кого больше мужчин или женщин?
select = """
    SELECT sex, COUNT(*) FROM titanik1 group by sex;
"""
cursor.execute(select)

In [75]:
cursor.fetchall()

[(0, 5), (1, 5)]

In [ ]:
#TODO Вопрос Какой процент мужчин выжил и какой процент женщин?

In [ ]:
#TODO Что вы можете сказать выживаемости пассижиров разного класса и пола?